In [35]:
import matplotlib.pyplot as plt
from sklearn import cluster, datasets
import numpy as np
import math

n_samples = 30
blobs,label = datasets.make_blobs(n_samples=n_samples,centers=5, random_state=10,cluster_std=[0.9,1.5,2.0,2.5,3])
data = np.array(blobs)

lbd = 0.1
delta = 1e-2
Lip = 1 + len(data) * lbd / delta
alpha = 1 / Lip
d = len(data[0])
n = len(data)
X = np.ones((n,d))*3.5  #initial value
# X = data #That is for my debugging process

In [36]:
def f(x):
    x = x.reshape((n, d))
    hg_1 = []
    for i in range(len(x)):
        x_1 = x[i, :] - x
        norm_x = np.linalg.norm(x_1, ord=2, axis=1, keepdims=True)
        hub_mat_1 = norm_x**2 / (2 * delta)
        hub_mat_2 = norm_x - 0.5 * delta
        hub_mat = np.where(norm_x > delta, hub_mat_2, hub_mat_1)
        hg_1.append(hub_mat[i:].sum())
    return (lbd * np.array(hg_1).sum() + np.linalg.norm(x-data, axis=1).sum()*0.5).flatten(order='A')



def hub_grad2(x, delta=1e-4):
    x = x.reshape((n, d))
    hg_1 = np.zeros((n, d))
    for i in range(len(x)):
        x_1 = x[i, :] - x
        norm_x = np.linalg.norm(x_1, ord=2, axis=1, keepdims=True)
        norm_x = np.where(norm_x > delta, norm_x, delta)
        x_1a = x_1 / norm_x
        x_1a = x_1a.sum(axis=0)
        hg_1[i, :] = lbd * x_1a
    return (hg_1 + x - data).flatten(order='A')

In [45]:
def L_BFGS(x, tol, maxit=20000):
    """
    L-BFGS
    """
    # initialize
    xk_1 = x.flatten(order='A')
    xk = xk_1
    m = 3
    k = 0
    stepsize = 1
    
    norm = np.zeros(maxit)  #store the norm of gk  
    norm[0] = np.linalg.norm(hub_grad2(xk).reshape((n, d)), ord=2)
    print(k, norm[k])

    #k=0 base case
    H_km = np.diag([1 for i in range(n*d)])   #H0_0
    # xk_list = np.array([xk-xk for i in range(m+1)])
    s_list = np.array([xk-xk for i in range(m)])
    y_list = np.array([xk-xk for i in range(m)])

    while True:       
        
        if np.linalg.norm(hub_grad2(xk).reshape((n, d)), ord=2) <= tol and k >= maxit:
            break        
        
        # approximation of hessian: H_km
        if k >= 1:
            # print(' 进入新的循环， ','iter: ', k) 
            xk_1 = xk
            xk = xk_1 + stepsize * dirc
            # print('stepsize * dirc: ', (stepsize * dirc).sum())
            norm[k] = np.linalg.norm(hub_grad2(xk).reshape((n, d)), ord=2)
            print(k, norm[k])

            sk_1 = xk - xk_1
            print('dis: ', (xk - X.flatten(order='A')).sum())
            yk_1 = hub_grad2(xk) - hub_grad2(xk_1)

            # print ('iter: ', k)
            for i in range(m-1):
                s_list[i] = s_list[i+1] #0 -> m-1, from older time to newer
                y_list[i] = y_list[i+1]
            
            s_list[-1] = sk_1
            y_list[-1] = yk_1
            # print('s_list:', s_list.sum())

            numerator = sk_1.T.dot(yk_1)  
            print('numerator: ', numerator)

            if numerator > 0 :       
                denominator = yk_1.T.dot(yk_1)

                if denominator == 0: iyp = 0
                else: iyp = numerator / denominator

                H_km = iyp * np.diag([1 for i in range(n*d)])           
            

                # print('iter: ', k, 'iyp: ', iyp)
            #numerator < 0: H_km unchanged
            
        
        #2-loop iteration for the descendant direciton
        q = hub_grad2(xk)

        
        
        alpha_list = np.zeros(m)
        for i in range(m, 0, -1): #3,2,1, idx-1
            si = np.array(s_list[i-1]) #s_l storing m si            
            yi = np.array(y_list[i-1])
            # print('i: ',i, 'si: ', si.sum())
            
            if yi.T.dot(si) == 0: alpha = 0                    
            else: alpha = (si.T.dot(q)) / (yi.T.dot(si))
            print('i: ',i, 'alpha: ', alpha)
                
            alpha_list[i-1] = alpha #from left to right and old to new

            q = q - alpha * yi

        # print('alpha list: ', alpha_list)

        r = (H_km.dot(q.T)).T
        for i in range(0, m):
            y_i = np.array(y_list[i])
            s_i = np.array(s_list[i])

            if (y_i.T.dot(s_i)) == 0: beta = 0
            else: beta = (y_i.T.dot(r)) / (y_i.T.dot(s_i))
                
            r = r + (alpha_list[i] - beta) * s_i
            # print("r inside: ", np.linalg.norm(r.reshape((n, d)), ord=2))
        
        dirc = - r
        # print("r outside: ", np.linalg.norm(dirc.reshape((n, d)), ord=2))

        #armijo on step size
        sigma = 0.005
        iter = 0
        while True:
            LHS = f(xk + stepsize * dirc) - f(xk)
            RHS = sigma * stepsize * hub_grad2(xk).T.dot(dirc)
            # print("#### Iteration for Armijo ####  ", iter, "stepsize: ", stepsize)
            # print("LHS: ", LHS, "RHS: ", RHS, 'LHS <= RHS: ', LHS <= RHS)
            if  LHS <= RHS or iter>20:
                break
            iter += 1
            stepsize = 0.5 * stepsize          
        
        k += 1

    return [xk, norm]

In [40]:
def agm(x0, tol, maxit=20000, **option):
    """
    Accelerated Gradient method
    """
    xk = x0.flatten(order='A')
    norm = np.zeros(maxit)  #store the norm of gk
    norm[0] = np.linalg.norm(hub_grad2(xk).reshape((n, d)), ord=2)
    y = xk
    alpha = 1 / Lip
    tk_1 = 1
    tk = 1
    print("----Accelerated Gradient Method----")
    print('ITER ;   G.NORM')
    for i in range(1, maxit):
        # alpha = BBstep_1(xk)
        ng = norm[i - 1]
        if ng <= tol:
            break
        xk, xk_1 = y - alpha * hub_grad2(y), xk
        norm[i] = np.linalg.norm(hub_grad2(xk).reshape((n, d)), ord=2)
        print(i, ng)
        tk, tk_1 = 0.5 * (1 + math.sqrt(1 + 4 * tk_1 ** 2)), tk
        beta = (tk_1 - 1) / tk
        y = xk + beta * (xk - xk_1)
    print("Local Minimizer x =", xk)
    norm = norm[0: i]
    return [xk, norm]

In [27]:
np.linalg.norm(hub_grad2(X).reshape((n, d)), ord=2)

70.0841867225643

In [ ]:
agm(X, tol = 1e-4, maxit=20000)

You can see from the log that the process simply stuck somewhere after several iteration. And since I started from the dataset itself so I assume it stuck on the starting point.

In [46]:
L_BFGS(X, tol = 1e-4, maxit=20000)

0 52.04735539489586
i:  3 alpha:  0
i:  2 alpha:  0
i:  1 alpha:  0
1 47.57137831665911
dis:  -0.0736405570703198
numerator:  0.11419467321510032
i:  3 alpha:  -6.963145184487904
i:  2 alpha:  0
i:  1 alpha:  0
2 47.571235626284775
dis:  -0.07390875718222345
numerator:  8.743161454474525e-09
i:  3 alpha:  -244595.0566522833
i:  2 alpha:  -5.86703533268244
i:  1 alpha:  0
3 47.55904196814693
dis:  -0.10571460609204708
numerator:  5.819603542699034e-05
i:  3 alpha:  -4339.215141324181
i:  2 alpha:  -17108.570288456944
i:  1 alpha:  -4.8052902022449215
4 47.55785328947673
dis:  -0.10989986149135067
numerator:  8.092943299269486e-07
i:  3 alpha:  -40978.5351980069
i:  2 alpha:  -1994.2340941727439
i:  1 alpha:  98943.48193278127
5 47.557853289476405
dis:  -0.109899861492468
numerator:  4.363031894596372e-26
i:  3 alpha:  -203098613090191.7
i:  2 alpha:  -11533.554879544357
i:  1 alpha:  317.33259694557063
6 47.55785328947632
dis:  -0.1098998614928286
numerator:  5.5788243217229955e-27
i:  

KeyboardInterrupt: 